<a href="https://colab.research.google.com/github/Suaif/Video_descriptor/blob/master/OurOwnFunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Con el objetivo de tener nuestro código más organizado, hemos decidido crear este documento `OurOwnFunctions.ipynb.` <br>
De esta forma podremos implementar este documento como un módulo en el documento principal `main.ipynb.`

---

**Lista de funciones implementadas:**

* `slice_video`
* `padding`
* `int2str`
* `clean_text`
* `decode_sequence`




In [ ]:
# Modulos necesarios para el correcto funcionamiento de las funciones.

import cv2
import os

import matplotlib.pyplot as plt

import numpy as np
import re    as re

from keras.preprocessing.sequence import pad_sequences
from google.colab.patches         import cv2_imshow


In [ ]:
def slice_video(video_id, pathIn, pathOut, height=240, width=240, numero_frames=6, format='.avi'):
 
  '''
  Dado un vídeo devuelve el `numero_frames` en la carpeta `pathOut` con 
  una `altura y anchura de (240x240)` en un formato por defecto `.avi`.
 
  -
  Parámetros:
    video_id         (string)  [Id del vídeo]
    pathIn           (string)  [Ruta de los vídeos]
    pathOut          (string)  [Ruta guardado/salida]
    height           (int)     [Altura del frame]
    width            (int)     [Anchura del frame]
    numero_frames    (int)     [Número de frames a dividir]
    format           (string)  [Formato del vídeo. '.*' ]
  '''

  # -- Parámetros
  vidcap     = cv2.VideoCapture(pathIn + video_id + format)
  max_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

  num_img, count = 1, 1
  dim = (width, height)

  # Mientras no sea divisible por numero_frames restamos uno a max_frames.
  while max_frames % numero_frames != 0:
     max_frames -= 1

  n_fps = max_frames / numero_frames

  # Mientras se siga pudiendo leer y count siga siendo menor que el número máximo de frames
  while vidcap.isOpened and count <= max_frames:

    vidcap.set(cv2.CAP_PROP_FPS, 1)    
    success,image = vidcap.read()
    
    # Hacemos resize de la imgen. 
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

    if count % n_fps == 0:
     
      # Si la ruta no existe, la crea.
      if not os.path.exists(pathOut):  
        os.mkdir(pathOut) 
     
      # Guarda la imagen almacenada en resized en la ruta dada.
      cv2.imwrite(pathOut + video_id + '_' + str(num_img) + ".jpg", resized)
      print(pathOut + video_id + '_' + str(num_img) + ".jpg")
      num_img += 1
    count += 1

In [ ]:
def padding(Y, maxlen=0, pad='post'):

  '''
  Rellena con ceros los espacios sobrantes de un ndarray.
  Además añade un 0 extra al final
  -
  Parámetros:
    Y       (ndarray) [Descripciones]
    maxlen  (int)     [Longitud máxima]
    pad     (string)  [Posición del padding]
  '''

  Yt = np.asarray(Y)
  Yt = Yt[:,np.newaxis]

  if maxlen == 0:
    
    maxlen = max([len(Yt[i].tolist()[0]) for i in range(len(Yt))]) 

  Ypad = np.zeros((len(Yt), maxlen+1))

  for i in range(len(Yt)):

    Ypad[i,:-1] = pad_sequences(Yt[i], maxlen=maxlen,  padding=pad, value=0.0)

  return Ypad

In [ ]:
def int2str(sec, tokenizer):

  '''
  Convierte una lista de enteros a una lista de palabras.

  -
  Parámetros:
    sec        (list)        [Lisa de enteros que se convertirán a palabras]
    tokenizer  (dictionary)  [Contiene eliccionrio para asignar palabras a enteros]
  '''

  #Si entra una lista se convierte a array
  if type(sec) is list:
    sec = np.asarray(sec)

  #Si solo hay una secuencia se añade otro eje
  if len(sec.shape) == 1:
    sec = sec[np.newaxis, :]

  nsecs, nints = sec.shape

  frases = np.zeros((nsecs), dtype=object) 

  for i in range(nsecs):
    frases[i] = sec[i].astype(int).tolist()

  return tokenizer.sequences_to_texts(frases)

In [ ]:
def clean_text(text):

  '''
  Limpia el texto pasado removiendo carácteres innecesarios.
  
  -
  Parámetros:
    text       (string) [Texto a limpiar]
  '''

  text = text.lower()

  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"what's", "that is", text)
  text = re.sub(r"where's", "where is", text)
  text = re.sub(r"how's", "how is", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"n't", " not", text)
  text = re.sub(r"n'", "ng", text)
  text = re.sub(r"'bout", "about", text)
  text = re.sub(r"'til", "until", text)
  text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
      
  return text

In [ ]:
def decode_sequence(input_seq):

  '''
  Utiliza el encoder y el decoder para obtener la secuencia de enteros
  correspondiente a la descripción.
  
  -
  Parámetros:
    input_seq       (array)[Secuencia de imágenes]
  '''

  
  # Hacemos predecir al encoder y sacamos los states
  states_value = encoder_model.predict(input_seq)

  # First character of the target sequence is the start character
  target_seq = [[2]]
  target_seq = np.asarray(target_seq)
  # print(target_seq.shape)

  # To simplify, here we assume a batch of size 1.
  stop_condition = False
  decoded_sentence = []
  while not stop_condition:
    #aqui hacemos predecir al decoder al que le metemos el target sequence mas el state que hemos sacado antes
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens)
    decoded_sentence.append(sampled_token_index) #sampled_char
    # print(sampled_token_index)

    # Exit condition: either hit max length or find stop character.
    if (len(decoded_sentence) > max_sentence_len) or (sampled_token_index==3):
        stop_condition = True

    # Update the target sequence (of length 1).    
    target_seq = [[sampled_token_index]]
    target_seq = np.asarray(target_seq)
    # print(target_seq)

    # Update states
    states_value = [h, c]

  return decoded_sentence